##https://github.com/openai/CLIP

In [ ]:
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
from torchvision.datasets import FGVCAircraft     #fjvc aircraft      هواپیما

import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [ ]:
transforms=transforms.Compose([     #تصاویر در زمان اموزش مدل 224 در 224 بودند پس ما هم این ری سایز رو  انجام میدیم
    transforms.Resize((224,224)),
                 transforms.ToTensor(),
                         transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225 ])       #
                           ])
#در ادامه تبدیل به تنسور میکنیم و با تجه به مقادیر لازم که در داکیومنت خود مدل هست نرمالایز میکنیم


train_set=FGVCAircraft(root="/content/drive/MyDrive/Colab Notebooks/data",split="train", transform=transforms,download=True)
#در اینجا داریم بخش ترین دیتاست رو دانلود میکنیم و ادرس دانلود ترنسفور های لازم رو انجام میدهیم


train_loader=DataLoader(train_set,batch_size=32,shuffle=True)
#استفاده از دیتا لودر برای تنظیم بچ و شافل تورو برای داده اموزس تا مدل بهتر اموزش ببینه


test_set=FGVCAircraft(root="/content/drive/MyDrive/Colab Notebooks/data",split="test", transform=transforms,download=True)
#در اینجا داریم بخش تست دیتاست رو دانلود میکنیم و ادرس دانلود ترنسفور های لازم رو انجام میدهیم

test_loader=DataLoader(train_set,batch_size=32,)
#استفاده از دیتا لودر برای تنظیم بچ و شافل نیازی نیست



100%|██████████| 2.75G/2.75G [01:56<00:00, 23.7MB/s]


Extracting /content/drive/MyDrive/Colab Notebooks/data/fgvc-aircraft-2013b.tar.gz to /content/drive/MyDrive/Colab Notebooks/data


In [ ]:
import clip


device="cuda"  if torch.cuda.is_available() else "cpu"
 #ایا جی پی یو وجود داره یا نه اگه داره روی جی پی یو اجرا شه در غیر این صورت از سی پی یو استفاده میکنیم

model, preprocess = clip.load('ViT-B/32', device)
#لود کردن مدل کلیپ(میتونه نسخه های  دیگشم باشه)


100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 103MiB/s]


#استفاده از انکودر تصویر کلیپ برای دسته بندی تصویر

In [ ]:
from tqdm import tqdm   #مخفف تقدم برای نمایش قشنگتر یک پروگرس بار


def get_features(data_loader):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(data_loader):
            features = model.encode_image(images.to(device))           #به ازای هر بچ تصاویر رو میفرستیم به دیوایس و انکد میکنه

            all_features.append(features)                   #ذخیره کردن ویژگی ها
            all_labels.append(labels)                       #ذخیره کردن لیبل ها

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
for x,y in train_loader:
  break

#خوندن یک بچ از دیتا

In [ ]:
train_set.classes                               #کلاس هاییه که داریم اسم هواپیماس

# len(train_set.classes) -> 100

['707-320',
 '727-200',
 '737-200',
 '737-300',
 '737-400',
 '737-500',
 '737-600',
 '737-700',
 '737-800',
 '737-900',
 '747-100',
 '747-200',
 '747-300',
 '747-400',
 '757-200',
 '757-300',
 '767-200',
 '767-300',
 '767-400',
 '777-200',
 '777-300',
 'A300B4',
 'A310',
 'A318',
 'A319',
 'A320',
 'A321',
 'A330-200',
 'A330-300',
 'A340-200',
 'A340-300',
 'A340-500',
 'A340-600',
 'A380',
 'ATR-42',
 'ATR-72',
 'An-12',
 'BAE 146-200',
 'BAE 146-300',
 'BAE-125',
 'Beechcraft 1900',
 'Boeing 717',
 'C-130',
 'C-47',
 'CRJ-200',
 'CRJ-700',
 'CRJ-900',
 'Cessna 172',
 'Cessna 208',
 'Cessna 525',
 'Cessna 560',
 'Challenger 600',
 'DC-10',
 'DC-3',
 'DC-6',
 'DC-8',
 'DC-9-30',
 'DH-82',
 'DHC-1',
 'DHC-6',
 'DHC-8-100',
 'DHC-8-300',
 'DR-400',
 'Dornier 328',
 'E-170',
 'E-190',
 'E-195',
 'EMB-120',
 'ERJ 135',
 'ERJ 145',
 'Embraer Legacy 600',
 'Eurofighter Typhoon',
 'F-16A/B',
 'F/A-18',
 'Falcon 2000',
 'Falcon 900',
 'Fokker 100',
 'Fokker 50',
 'Fokker 70',
 'Global Express

In [ ]:
x.shape

torch.Size([32, 3, 224, 224])

سی دوتا تصویر 3 کاناله با ابعاد (ری سایز شده) دویست بیست چهار در دویست بیست چهار

In [ ]:
y.shape

torch.Size([32])

سی و دوتا عدده که هر کدوم مربوط به یک کلاسه

In [ ]:
f=model.encode_image(x.to(device))
f.shape

torch.Size([32, 512])

سی و دوتا تصویر که از هر کدوم 512 تا ویژگی استخراج کرده برای هر تصویر یک بردار امبدینگ 512 بعدی داریم

In [ ]:
# Calculate the image features
#محاسبه ویژگی تصاویر برای هر دو مجموعه تست و ترین

train_features, train_labels = get_features(train_loader)
test_features, test_labels = get_features(test_loader)

100%|██████████| 105/105 [01:01<00:00,  1.72it/s]


In [ ]:
print(train_features.shape)

#3334 تصویر
#به ازای هر کدوم یک بردار 512 بعدی امبدینگ


print(test_features.shape)


#3334 تصویر
#به ازای هر کدوم یک بردار 512 بعدی امبدینگ


(3334, 512)
(3334, 512)


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression


# Perform logistic regression
#استفاده از مدل لاجستیک رگرسیون (مدل یادگیری ماشین) و فیت کردن روی خروجی انکودر مدل کلیپ
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)
#از مدل های دیگر یادگیری ماشین برای دسته بندی همچون درخت تصمیم جنگل تصادفی و ... استفاده کرد

# Evaluate using the logistic regression classifier
#گرفتن خروجی (پیش بینی برچسب کلاس) از داده تست و به دست اوردن اکیوریسی و دقت مدل
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 91.422


#zero shot learning

In [27]:
#غکس:   /content/drive/MyDrive/Colab Notebooks/data/giraffe_reticulated_IS.jpg

from PIL import Image         #کتابخانه پیل برای خوندن عکس

image= Image.open("/content/drive/MyDrive/Colab Notebooks/data/giraffe_reticulated_IS.jpg")

In [28]:
image_input= preprocess(image).unsqueeze(0).to(device)

#پریپاسس مدل کلیپ      یک بعد با دستور ان اس کوییز بهش اضافه میکنیم که بهش یک تصویر سه کاناله دویست بیست چهار در دویست بیست چهار

الان تصویر امادس برای گرفتن بردار امبدینگ یا همون ویژگی از مدل

In [39]:
texts =["giraffe","bufallo","tree","a giraffe", "a photo of a giraffe"," a giraffe eating tree","a boy playing football"]
#میخاییم شباهت این تصویر رو با این سه تا متن بسنجیم


text_input=  torch.cat([clip.tokenize(f"{c}") for c in texts]).to(device)
# توکنایز کردن متن و بردن روی دیوایس

In [40]:
# Calculate features
#محاسبه ویژگی های تصویر و متن

with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_input)

In [41]:
image_features.shape , text_features.shape

(torch.Size([1, 512]), torch.Size([7, 512]))

یک تصویر با یک بردار ویژگی 512 تایی

هفت تصویر هر کدام بردار ویژگی 512 تایی

In [42]:
#سنجش فاصله با استفاده از ضرب داخلی

#اول ویژگی های تصویر رو نرمالایز میکنه و بعد تقسیم میکنه ایمیج فیچر رو به این عدد و این تبدیل میشه به یک بردار نرمالایزه
image_features /= image_features.norm(dim=-1, keepdim=True)
#به همین ترتیب برای متن
text_features /= text_features.norm(dim=-1, keepdim=True)

#حالا که نرمالیزه شدن وقتی ضرب داخلی کنیم(میزان شباهت رو نشون میده) و از سافت مکس ردش کنیم درصد احتمال رو نشون میده
similarity = (100.0 * image_features @ text_features.T)#.softmax(dim=-1)


for i in range(len(texts)):
  print(f"{texts[i]}: {similarity[0][i]}")


giraffe: 31.640625
bufallo: 23.015625
tree: 22.265625
a giraffe: 32.0625
a photo of a giraffe: 33.25
 a giraffe eating tree: 32.09375
a boy playing football: 20.90625


این برا این حالت بود که ما یک تصویر رو با متن مقایسه کنیم

حالا اگه بخاییم دسته بندی انجام بدیم چی ؟ باید برای هر کلاسمون یک متن بسازیم و اون متن هارو بدیم به مدل بردار ویژگی استخراج کنه و با هر تصویر مقایسه کنیم

#استفاده از زیرو شات  برای دسته بندی

In [53]:
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in train_set.classes]).to(device)

predictin=np.zeros((0))
Labels=np.zeros((0))

with torch.no_grad():
  text_features = model.encode_text(text_inputs)
  text_features /= text_features.norm(dim=-1, keepdim=True)
  for image,labels in test_loader:
      image_features = model.encode_image(image.to(device))

      image_features /= image_features.norm(dim=-1, keepdim=True)

      similarity = (image_features @ text_features.T)

      #پیش بینی اینکه کدوم کلاس بیشترین شباهت رو داشت
      pred=np.argmax(similarity.cpu().numpy(),axis=1)

      #اذغام کردن اون ماتریس خالی با ماتریس پیش بینی مون
      prediction=np.concatenate((predictin,pred))

      #اذغام کردن اون ماتریس خالی با ماتریس برچسب مون
      test_labels=np.concatenate((Labels,labels))



In [55]:
accuracy = np.mean((test_labels == prediction).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 0.000


In [59]:
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in train_set.classes]).to(device)

predictin=np.zeros((0))
Labels=np.zeros((0))

with torch.no_grad():
  text_features = model.encode_text(text_inputs)
  text_features /= text_features.norm(dim=-1, keepdim=True)
  for image,labels in test_loader:
      image_features = model.encode_image(image.to(device))

      image_features /= image_features.norm(dim=-1, keepdim=True)

      similarity = (image_features @ text_features.T)

      #پیش بینی اینکه کدوم کلاس بیشترین شباهت رو داشت
      pred=np.argmax(similarity.cpu().numpy(),axis=1)

      #اذغام کردن اون ماتریس خالی با ماتریس پیش بینی مون
      prediction=np.concatenate((predictin,pred))

      #اذغام کردن اون ماتریس خالی با ماتریس برچسب مون
      test_labels=np.concatenate((Labels,labels))


# Get unique labels from predictions and convert them to integers
unique_labels = np.unique(prediction).astype(int)

# Filter target names to match predicted labels
filtered_target_names = [train_set.classes[i] for i in unique_labels]


from sklearn.metrics import classification_report

# Pass the filtered target names to classification_report
print(classification_report(test_labels, prediction, target_names=filtered_target_names, labels=unique_labels)) # Pass labels to specify the classes to include

              precision    recall  f1-score   support

  Fokker 100       0.00      0.00      0.00       0.0
   Fokker 70       0.00      0.00      0.00       0.0
       MD-90       0.00      0.00      0.00       0.0
   Saab 2000       0.00      0.00      0.00       0.0
    Saab 340       0.00      0.00      0.00       0.0
      Tu-154       0.00      0.00      0.00       0.0

   micro avg       0.00      0.00      0.00       0.0
   macro avg       0.00      0.00      0.00       0.0
weighted avg       0.00      0.00      0.00       0.0



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: 

اینجا دقتی که از زیرو شات میگیریم کمتره چون اسم کلاس ها عدد و چیزای مفهمومی نیست البته میتونیم با تنظیم پرامپت بهتر نتیجه بهتر بگیریم

زیرو شات جاهایی که اسم کلاس هامون مفهومیه بدرد بخوره

#https://github.com/lucidrains/DALLE2-pytorch

نحوه پیاده سازی مدل دال ای و اموزش دیگودر و پیرور و مدل های پری ترین ان و چک پوینت ها در هاگینگ فیس